In [1]:
import math
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import sys
import seaborn as sns

from PIL import Image

# Import required libraries  
import os  
import json  
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    HnswAlgorithmConfiguration,
    HnswParameters, 
    SearchField,  
    SearchFieldDataType,
    SearchableField,  
    SearchIndex,  
    SemanticPrioritizedFields,
    SemanticSearch,
    SemanticConfiguration,  
    SemanticField,  
    SimpleField, 
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,  
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = 'https://ai-search-msft.search.windows.net' #ai search url
index_name = 'test-wan' 
key = 'niAQ9XHhtWzOCNuFo7Zeu4yVhHAj9D2cLiOnVHQkBkAzSeBtak0R' #ai search key
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)

In [2]:
computer_vision_endpoint = "https://computer-vision-msft.cognitiveservices.azure.com"
computer_vision_version = "?api-version=2023-02-01-preview"

vec_img_url = computer_vision_endpoint + "/computervision/retrieval:vectorizeImage" + computer_vision_version  # For doing the image vectorization

computer_vision_headers = {
    'Content-type': 'application/json',
    'Ocp-Apim-Subscription-Key': 'd395f4b6ace44e64873f522b4c507630'
}

In [3]:
client = AzureOpenAI(
  api_key = 'cca62c238bb542cd9bda2087e505c92e',  
  api_version = "2024-02-15-preview",
  azure_endpoint = "https://aistudiomsft9822613278.openai.azure.com/"
)


In [4]:
def image_embedding(imageurl):
    """
    Embedding image using Azure CV 4.0
    """
    image = {"url": imageurl}
    print(f'image : {image}')
    print(f'vec_img_url : {vec_img_url} ')
    r = requests.post(vec_img_url, data=json.dumps(image), headers=computer_vision_headers)
    print(r)
    image_emb = r.json()['vector']

    return image_emb

In [5]:
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [6]:
urls = [
    "https://github.com/wangil/github_tutorial/blob/main/MSFT/abibag.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/abibas.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/abibos.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/adadas.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/adidos.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/adimas.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/daiads.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/didasa.jpg?raw=true"
    , "https://github.com/wangil/github_tutorial/blob/main/MSFT/sports.jpg?raw=true"
]

In [7]:
print(urls[0].split('/')[-1].split('.')[0])

abibag


In [19]:
image_all_list = []
url_dict = {}

i = 1
for url in urls:
    url_dict["id"] = str(i)
    url_dict['title'] = url.split('/')[-1].split('.')[0]
    url_dict['content'] = url
    url_dict['category'] = "logo"
    image_all_list.append(url_dict)
    url_dict = {}
    i += 1

In [20]:
image_all_list

[{'id': '1',
  'title': 'abibag',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibag.jpg?raw=true',
  'category': 'logo'},
 {'id': '2',
  'title': 'abibas',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibas.jpg?raw=true',
  'category': 'logo'},
 {'id': '3',
  'title': 'abibos',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibos.jpg?raw=true',
  'category': 'logo'},
 {'id': '4',
  'title': 'adadas',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/adadas.jpg?raw=true',
  'category': 'logo'},
 {'id': '5',
  'title': 'adidos',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/adidos.jpg?raw=true',
  'category': 'logo'},
 {'id': '6',
  'title': 'adimas',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/adimas.jpg?raw=true',
  'category': 'logo'},
 {'id': '7',
  'title': 'daiads',
  'content': 'https://github.com/wangil/github_tutorial/blob/main/

In [21]:
client = AzureOpenAI(
  api_key = 'cca62c238bb542cd9bda2087e505c92e',  
  api_version = "2024-02-15-preview",
  azure_endpoint = "https://aistudiomsft9822613278.openai.azure.com/"
)

i=1
# Generate embeddings for title and content fields
for item in image_all_list:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = image_embedding(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings



image : {'url': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibag.jpg?raw=true'}
vec_img_url : https://computer-vision-msft.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview 
<Response [200]>
image : {'url': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibas.jpg?raw=true'}
vec_img_url : https://computer-vision-msft.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview 
<Response [200]>
image : {'url': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibos.jpg?raw=true'}
vec_img_url : https://computer-vision-msft.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-preview 
<Response [200]>
image : {'url': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/adadas.jpg?raw=true'}
vec_img_url : https://computer-vision-msft.cognitiveservices.azure.com/computervision/retrieval:vectorizeImage?api-version=2023-02-01-pr

In [22]:
image_all_list[0]

{'id': '1',
 'title': 'abibag',
 'content': 'https://github.com/wangil/github_tutorial/blob/main/MSFT/abibag.jpg?raw=true',
 'category': 'logo',
 'titleVector': [-0.02874141000211239,
  -0.024963397532701492,
  0.0026036209892481565,
  -0.018334055319428444,
  -0.02135646529495716,
  0.012538725510239601,
  -0.01843385025858879,
  -0.009844218380749226,
  -0.002256115199998021,
  -0.010421612299978733,
  0.022511253133416176,
  0.007705720607191324,
  -0.0005573459784500301,
  -0.0010095491306856275,
  0.002690942957997322,
  -0.009951143525540829,
  0.03552757576107979,
  0.02457846887409687,
  0.018376825377345085,
  -0.015482723712921143,
  -0.008532606065273285,
  0.02939721569418907,
  0.006825372111052275,
  -0.003480405081063509,
  -0.023452192544937134,
  0.0032184391748160124,
  0.005421091802418232,
  -0.026902303099632263,
  0.004023939836770296,
  -0.022211864590644836,
  0.009751549921929836,
  -0.015297387726604939,
  -0.021798420697450638,
  -0.008268858306109905,
  -0.0

In [23]:
# Output embeddings to docVectors.json file
with open("docVectors.json", "w") as f:
    json.dump(image_all_list, f)

In [26]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1024, vector_search_profile_name="myHnswProfile"),
]


# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        )
    ]
)


semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)

result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 test-wan created


In [27]:
# Upload some documents to the index
with open('docVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 9 documents
